### <b> Feature engineering </b> notebook (Nr 2) for AmesHousingPrices
by Martijn de Vries </br>
martijndevries91@gmail.com <br>
http://github.com/martijndevries/AmesHousingPrices

## Problem Statement

A real estate company in Ames, Iowa is looking for a new and improved way to evaluate the market value of a house. Using the Ames data set as training data, I will build a predictive linear regression model to predict the sale price of a house.

To gauge the model performance, I will compare my results against a 'benchmark model', which is a simple OLS regression of total living area versus sale price. How much can a more complex model improve over this simple basic model? I will try out different models with different numbers of features using different linear regression techniques, and ultimately identify which model does the best job at predicting the market value of the house. To evaluate and compare models, I will use the r2-score, as well as the root mean squared error (RMSE). 

## In this notebook

Contains all the feature engineering functions that are used to make the model inputs. For the full feature engineering exploration, see the feature_engineering_exploration notebook

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import sys

### Feature engineering function for Model 1

First let's make some helper functions that we can re-use for the other models

In [2]:
def base_feature_engineer(df_train, df_test):
    """
    Basic feature engineering steps we'll take for every model
    Parameters:
        1) Ames training data set
        2) Ames testing data set
    Returns:
        1) Feature engineered Ames training data 
        2) Feature engineered Ames testing data
    """
    
    #For the train dataframe, drop the two outliers with huge living areas
    inds = df_train[df_train['Gr Liv Area'] > 4000].index
    df_train.drop(index=inds, inplace=True)
    
    df_train['tot_area'] = df_train['Gr Liv Area'] + df_train['Total Bsmt SF']
    df_test['tot_area'] = df_test['Gr Liv Area'] + df_test['Total Bsmt SF']
    
    #Make home functionality categories more coarse
    func_dic = {'Typ':'Typ', 'Min1':'Min', 'Min2':'Min', 'Mod':'Mod', 'Maj1':'Maj', 'Maj2':'Maj', 'Sev':'Sev', 'Sal':'Sev'}
    df_train['Func'] = df_train['Functional'].map(lambda func: func_dic[func])
    df_test['Func'] = df_test['Functional'].map(lambda func: func_dic[func])
    
    #drop the A (agr) MS zoning category, because this category is not in the test data anyway and it removes an outlier from the train data
    agr_inds = df_train[df_train['MS Zoning'] == 'A (agr)'].index
    df_train.drop(index=agr_inds, inplace=True)

    return df_train.copy(), df_test.copy()

In [3]:
def calc_adj_gar(row, type_coefs, qual_coefs):
    """
    calculate adjusted garage area based on 
    1) the actual square footage
    2) the garage quality
    3) the garage type 
    the type_coefs and qual_coefs should be pandas series, so that I can look up the appropriate coefficient based on input quality and type
    """
    gtype = row['Garage Type']
    area = row['Garage Area']
    gqual = row['Garage Qual']
    return area * (type_coefs[gtype] + qual_coefs[gqual])/2


In [4]:
def make_adj_gar(df_train, df_test):
    """
    Create adjusted gar area columns in the train and test data
    Parameters:
        1) Ames training data set
        2) Ames testing data set
    Returns:
        1) Ames training data set with gar_adj_area column
        2) Ames testing data with gar_adj_area garage column
    """
    
    #calculate the garage area, adjusted for garage quality and type - using log of the saleprice here to calculate the coefficients
    #since I'll be using log-transforms
    df_train_gq = np.log(df_train.groupby(by="Garage Qual")['SalePrice'].mean())
    gar_qual_coefs = (df_train_gq-df_train_gq['NP'])/(df_train_gq['TA'] -df_train_gq['NP'])
    
    df_train_gt = np.log(df_train.groupby(by="Garage Type")['SalePrice'].mean())
    gar_type_coefs = (df_train_gt-df_train_gt['NP'])/(df_train_gt['Attchd'] -df_train_gt['NP'])

    gar_adj_train_l, gar_adj_test_l = [],[]
    for rowind in df_train.index:
        row = df_train.loc[rowind]
        gar_adj_train_l.append(calc_adj_gar(row, gar_type_coefs, gar_qual_coefs))
    df_train['gar_adj_area'] = gar_adj_train_l
    
    for rowind in df_test.index:
        row = df_test.loc[rowind]
        gar_adj_test_l.append(calc_adj_gar(row, gar_type_coefs, gar_qual_coefs))
    df_test['gar_adj_area'] = gar_adj_test_l
    
    return df_train.copy(), df_test.copy()

In [5]:
def dummify_and_select(df_train, df_test, features_dum, features_baseline, features_sel):
    """
    Dummify and select appropriate features for this model
    Parameters:
        1) Ames training data set
        2) Ames testing data set
        3) List of features (colnames) to dummify
        4) list of feature baselines to drop (Coding these by hand because I would like the most 'normal' value to be dropped, eg. 'typical' or 'average' or '3 out of 5')
        4) List of non-dummy features (colnames) to select
    Returns:
        1) Ames model input for training set
        2) Ames model input for testing set
    """
        
    df_train_dummies = pd.get_dummies(df_train, columns=features_dum)
    df_train_dummies.drop(columns=features_baseline, inplace=True) #drop baseline (most common occurence)
    
    df_test_dummies = pd.get_dummies(df_test, columns=features_dum)
    df_test_dummies.drop(columns=features_baseline, inplace=True) #drop baseline (most common occurence)
     
    #add the dummified colnames to the features to select
    for col in df_train_dummies.columns:
        for feature in features_dum:
            if feature + '_' in col:
                features_sel.append(col)
            
    #add dummy columns that are in the train data but not the test data
    for feature in features_sel:
        try:
            df_test_dummies[feature]
        except:
            df_test_dummies[feature] = 0
    
    
    df_model_test = df_test_dummies[features_sel]
    
    #saleprice only goes in the training set
    features_sel.append('SalePrice')
    df_model_train = df_train_dummies[features_sel]
    
    return df_model_train, df_model_test

Then on to the main function for model 1

In [6]:
def feature_engineer_m1(df_train, df_test):
    """
    Feature engineer and extract the features that we want to fit in our model, for both the train and the test data
    These are 
        1) the total living area (based on above grade and basement living area),
        2) the garage area (adjusted by garage quality and garage type)
        3) the overall quality of the house
        4) the year built
        5) the year the house was remodeled or added to
        6) the home functionality
        7) the Zoning code of the building
    Parameters:
        1) the Ames training dataframe
        2) the Ames testing dataframe
    Returns:
        1) Ames training model input (with columns listed above)
        2) Ames testing model input (with columns listed above)
    """
    
    df_train, df_test = base_feature_engineer(df_train, df_test)
    df_train, df_test = make_adj_gar(df_train, df_test)

    #features to dummify and otherwise select for model 1
    features_dum = ['MS Zoning', 'Func']
    features_baseline = ['MS Zoning_RL', 'Func_Typ'] #the baseline values I should drop
    features_sel = ['Id', 'tot_area', 'gar_adj_area', 'Year Built', 'Year Remod/Add', 'Overall Qual', 'Overall Cond']   
    
    df_model_train, df_model_test = dummify_and_select(df_train, df_test, features_dum, features_baseline, features_sel)
    
    #rename columns
    rename_dict = {'Year Built':'yr_built', 'Year Remod/Add':'yr_remod', 'Overall Qual':'qual'}
    df_model_train = df_model_train.rename(columns=rename_dict)
    df_model_test = df_model_test.rename(columns=rename_dict)
    
    return df_model_train, df_model_test

In [7]:
#Reload train and test data and feed through the feature engineering function
df_train = pd.read_csv('../datasets/train_cleaned.csv', dtype={'MS SubClass':'object'})
df_test = pd.read_csv('../datasets/test_cleaned.csv', dtype={'MS SubClass':'object'})

df_fe_train, df_fe_test = feature_engineer_m1(df_train, df_test)

df_fe_train.to_csv('../model_inputs/train_engineered_m1.csv', index=False)
df_fe_test.to_csv('../model_inputs/test_engineered_m1.csv', index=False)

## Feature Engineering for model 2

This model adds a few extra columns

In [8]:
def make_adj_fireplace(df_train, df_test):
    """
    Create adjusted fireplace columns in the train and test data
    Parameters:
        1) Ames training data set
        2) Ames testing data set
    Returns:
        1) Ames training data set with adj_fireplace column
        2) Ames testing data with adj_fireplace column
    """
    
    df_train_fp = np.log(df_train.groupby(by="Fireplace Qu")['SalePrice'].mean())
    fp_qual_coefs = (df_train_fp-df_train_fp['NP'])/(df_train_fp['TA'] - df_train_fp['NP'])
    
    adj_fp_train_l, adj_fp_test_l = [],[]
    for rowind in df_train.index:
        row = df_train.loc[rowind]
        qual = row['Fireplace Qu']
        adj_fp_train_l.append(row['Fireplaces'] * fp_qual_coefs[qual]) 
    df_train['adj_fireplace']  = adj_fp_train_l
    
    for rowind in df_test.index:
        row = df_test.loc[rowind]
        qual = row['Fireplace Qu']
        adj_fp_test_l.append(row['Fireplaces'] * fp_qual_coefs[qual]) 
    df_test['adj_fireplace']  = adj_fp_test_l
    
    return df_train.copy(), df_test.copy()

In [9]:
def make_adj_vnr(df_train, df_test):
    """
    Create adjusted mas_vnr_area columns in the train and test data
    Parameters:
        1) Ames training data set
        2) Ames testing data set
    Returns:
        1) Ames training data set with adj_mas_vnr_area column
        2) Ames testing data with adj_mas_vnr_area column
    """
    
    df_train_mv = np.log(df_train.groupby(by="Mas Vnr Type")['SalePrice'].mean())
    mv_qual_coefs = (df_train_mv-df_train_mv['None'])/(df_train_mv['BrkFace'] - df_train_mv['None'])
    
    #there is a veneer category in the test data that does not exst in the train data
    #since it's only 1 entry, I'm just going to cheat and set the coefficient to 1
    mv_qual_coefs['CBlock'] = 1

    adj_mv_train_l, adj_mv_test_l = [],[]
    for rowind in df_train.index:
        row = df_train.loc[rowind]
        vtype = row['Mas Vnr Type']
        adj_mv_train_l.append(row['Mas Vnr Area'] * mv_qual_coefs[vtype]) 
    df_train['adj_mas_vnr_area']  = adj_mv_train_l
    
    for rowind in df_test.index:
        row = df_test.loc[rowind]
        vtype = row['Mas Vnr Type']
        adj_mv_test_l.append(row['Mas Vnr Area'] * mv_qual_coefs[vtype]) 
    df_test['adj_mas_vnr_area']  = adj_mv_test_l
    
    return df_train.copy(), df_test.copy()

In [10]:
def feature_engineer_m2(df_train, df_test):
    """
    Feature engineer and extract the features that we want to fit in our model, for both the train and the test data
    This model uses the following columns:
        1) the total living area (based on above grade and basement living area),
        2) the garage area (adjusted by garage quality and garage type)
        3) the overall quality of the house
        4) the year built, 
        5) the year the house was remodeled or added to, 
        6) the home functionality, 
        7) the Zoning code of the building
        8) the total nr of rooms aboveground, 
        9) the number of full bathrooms
        10) the adjusted masonry veneer area
        11) the adjusted number of fireplaces
        12) the neighborhood
    Parameters:
        1) the Ames training dataframe
        2) the Ames testing dataframe
    Returns:
        1) Ames training model input (with columns listed above)
        2) Ames testing model input (with columns listed above)
    """

    df_train, df_test = base_feature_engineer(df_train, df_test)
    
    #adjusted columns
    df_train, df_test =  make_adj_gar(df_train, df_test)
    df_train, df_test = make_adj_fireplace(df_train, df_test)
    df_train, df_test = make_adj_vnr(df_train, df_test)

    features_dum = ['MS Zoning', 'Func', 'Neighborhood']
    features_baseline = ['MS Zoning_RL', 'Func_Typ', 'Neighborhood_NAmes']
    features_sel = ['Id', 'tot_area', 'gar_adj_area', 'Year Built', 'Year Remod/Add', 'Overall Qual', \
              'TotRms AbvGrd', 'Full Bath', 'adj_mas_vnr_area', 'adj_fireplace']   
    df_model_train, df_model_test = dummify_and_select(df_train, df_test, features_dum, features_baseline, features_sel)
 
    
    #rename columns
    rename_dict = {'Year Built':'yr_built', 'Year Remod/Add':'yr_remod', 'Overall Qual':'qual', 'TotRms AbvGrd':'tot_rooms_abv',\
                   'Full bath':'full_bath'}
    df_model_train = df_model_train.rename(columns=rename_dict)
    df_model_test = df_model_test.rename(columns=rename_dict)
    
    return df_model_train, df_model_test

In [11]:
#Reload train and test data and feed through the feature engineering function
df_train = pd.read_csv('../datasets/train_cleaned.csv', dtype={'MS SubClass':'object'})
df_test = pd.read_csv('../datasets/test_cleaned.csv', dtype={'MS SubClass':'object'})

df_fe_train, df_fe_test = feature_engineer_m2(df_train, df_test)

df_fe_train.to_csv('../model_inputs/train_engineered_m2.csv', index=False)
df_fe_test.to_csv('../model_inputs/test_engineered_m2.csv', index=False)

## Model 3

Stepping back for a moment: I would like to be able to see if the model I made with the adjusted numbers is actually better than the a model with 'unadjusted' columns, eg. if I just include the Garage area, Mas vnr area, and number of fireplaces. We will call this 'model 3'.

In [12]:
def feature_engineer_m3(df_train, df_test):
    """
    Feature engineer and extract the features for model 3, for both the train and the test data
    This model uses the following columns:
        1) the total living area (based on above grade and basement living area),
        2) the garage area 
        3) the overall quality of the house
        4) the year built, 
        5) the year the house was remodeled or added to, 
        6) the home functionality, 
        7) the Zoning code of the building
        8) the total nr of rooms aboveground, 
        9) the number of full bathrooms
        10) the masonry veneer area
        11) thenumber of fireplaces
        12) the neighborhood
    Parameters:
        1) the Ames training dataframe
        2) the Ames testing dataframe
    Returns:
        1) Ames training model input (with columns listed above)
        2) Ames testing model input (with columns listed above)
    """

    df_train, df_test = base_feature_engineer(df_train, df_test)
    
    features_dum = ['MS Zoning', 'Func', 'Neighborhood']
    features_baseline = ['MS Zoning_RL', 'Func_Typ', 'Neighborhood_NAmes']
    features_sel = ['Id', 'tot_area', 'Garage Area', 'Year Built', 'Year Remod/Add', 'Overall Qual', \
                    'TotRms AbvGrd', 'Full Bath', 'Mas Vnr Area', 'Fireplaces']   

    df_model_train, df_model_test = dummify_and_select(df_train, df_test, features_dum, features_baseline, features_sel)
    
    #rename columns
    rename_dict = {'Year Built':'yr_built', 'Year Remod/Add':'yr_remod', 'Overall Qual':'qual', 'TotRms AbvGrd':'tot_rooms_abv',\
                           'Full bath':'full_bath', 'Mas Vnr Area':'mas_vnr_area', 'Fireplaces':'fireplaces'}
    df_model_train = df_model_train.rename(columns=rename_dict)
    df_model_test = df_model_test.rename(columns=rename_dict)
    
    return df_model_train, df_model_test

In [13]:
#Reload train and test data and feed through the feature engineering function
df_train = pd.read_csv('../datasets/train_cleaned.csv', dtype={'MS SubClass':'object'})
df_test = pd.read_csv('../datasets/test_cleaned.csv', dtype={'MS SubClass':'object'})

df_fe_train, df_fe_test = feature_engineer_m3(df_train, df_test)

print(df_fe_train.shape, df_fe_test.shape)

df_fe_train.to_csv('../model_inputs/train_engineered_m3.csv', index=False)
df_fe_test.to_csv('../model_inputs/test_engineered_m3.csv', index=False)

(2047, 47) (878, 46)


## Model 4

In the modeling notebook, it seems that Model 3 is actually better than Model 2. Additionally. it looks like the r2 scores for the training and validation data have been very similar, indicating that we are not at the overfitting point yet. It seems that we can still add more features to the model before we reach the overfitting stage.

In [14]:
def dummify_condition(df_train, df_test):
    """
    Dummify the 'Condition 1' and 'Condition 2' columns together
    Parameters:
        1) the Ames training dataframe
        2) the Ames testing dataframe
    Returns:
        1) Ames training model input (with dummified Condition columns named condit_')
        2) Ames testing model input (with dummified Condition columns named condit_')
    """
    all_conds = df_train['Condition 1'].unique().tolist()
    all_conds2 = df_train['Condition 2'].unique().tolist()
    
    #Drop Norm from the list - that's the baseline
    all_conds.remove('Norm')
    #Loop over all conditions, make a dummy column for that condition, and set val to 1 if the condition is in either the condition 1 or 2 columns
    #I found this way of using np.where on stackoverflow: https://stackoverflow.com/questions/61550532/dummy-variable-from-two-columns-in-python
    for cond in all_conds:
        colname = 'condit_' + cond
        if cond in all_conds2:
            df_train[colname] = np.where((df_train['Condition 1'] == cond) | (df_train['Condition 2'] == cond), 1, 0)
            df_test[colname] = np.where((df_test['Condition 1'] == cond) | (df_test['Condition 2'] == cond), 1, 0)
        else:
            df_train[colname] = np.where((df_train['Condition 1'] == cond), 1, 0)
            df_test[colname] = np.where((df_test['Condition 1'] == cond), 1, 0)    

    return df_train.copy(), df_test.copy()

In [15]:
def feature_engineer_m4(df_train, df_test):
    """ 
    Feature engineer and extract the features for model 3, for both the train and the test data
    This model uses the following columns:
        1) the total living area (based on above grade and basement living area),
        2) the garage area 
        3) the overall quality of the house
        4) the year built, 
        5) the year the house was remodeled or added to, 
        6) the home functionality, 
        7) the Zoning code of the building
        8) the total nr of rooms aboveground, 
        9) the number of full bathrooms
        10) the masonry veneer area
        11) thenumber of fireplaces
        12) the neighborhood
        13) Basement quality
        14) Central air 
        15) Paved drive
        16) Year sold
        17) Lot Frontage
        18 ) Garage Cars
    Parameters:
        1) the Ames training dataframe
        2) the Ames testing dataframe
    Returns:
        1) Ames training model input (with columns listed above)
        2) Ames testing model input (with columns listed above)
    """

    df_train, df_test = base_feature_engineer(df_train, df_test)
    
    df_train, df_test = dummify_condition(df_train, df_test)
    
    #change 'Yr Sold' to object to dummify
    df_train['Yr Sold'] = df_train['Yr Sold'].astype('object')
    df_test['Yr Sold'] = df_test['Yr Sold'].astype('object')
    
    #now dummify the other columns and select using the dummify_and_select function
    features_dum = ['MS Zoning', 'Func', 'Neighborhood', 'Kitchen Qual', 'Bsmt Qual', 'Central Air', 'Alley', 'Paved Drive', 'Yr Sold']
    
    #the baseline categories to drop for each of the above columns
    col_baselines = ['RL', 'Typ', 'NAmes', 'TA', 'TA', 'Y', 'NP', 'Y', '2006']
    features_baseline = []
    for i,col in enumerate(features_dum):
        features_baseline.append(col + '_' + col_baselines[i])
    
    features_sel = ['Id', 'tot_area', 'Garage Area', 'Lot Frontage', 'Year Built', 'Year Remod/Add', 'Overall Qual', \
              'TotRms AbvGrd', 'Full Bath', 'Mas Vnr Area', 'Fireplaces', 'Overall Cond', 'Garage Cars'] 
    
    #add the already-dummified 'condit_columns'
    for colname in df_train.columns:
        if 'condit_' in colname:
            features_sel.append(colname)

    df_model_train, df_model_test = dummify_and_select(df_train, df_test, features_dum, features_baseline, features_sel)

    #rename columns
    rename_dict = {'Year Built':'yr_built', 'Garage Area':'gar_area', 'Lot Frontage':'lot_frontage','Year Remod/Add':'yr_remod', \
                   'Garage Cars':'gar_cars','Overall Qual':'qual', 'TotRms AbvGrd':'tot_rooms_abv',\
                    'Full Bath':'full_bath', 'Mas Vnr Area':'mas_vnr_area', 'Fireplaces':'fireplaces', 'Overall Cond':'cond'}
    df_model_train = df_model_train.rename(columns=rename_dict)
    df_model_test = df_model_test.rename(columns=rename_dict)
    
    return df_model_train, df_model_test

In [16]:
#Reload train and test data and feed through the feature engineering function
df_train = pd.read_csv('../datasets/train_cleaned.csv', dtype={'MS SubClass':'object'})
df_test = pd.read_csv('../datasets/test_cleaned.csv', dtype={'MS SubClass':'object'})

df_fe_train, df_fe_test = feature_engineer_m4(df_train, df_test)
print(df_fe_train.shape, df_fe_test.shape)
df_fe_train.to_csv('../model_inputs/train_engineered_m4.csv', index=False)
df_fe_test.to_csv('../model_inputs/test_engineered_m4.csv', index=False)

(2047, 75) (878, 74)


/var/folders/w4/w_knz6hd3w7gjmkp2lg297yr0000gr/T/ipykernel_97028/2502697827.py:15: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_train_dummies = pd.get_dummies(df_train, columns=features_dum)
/var/folders/w4/w_knz6hd3w7gjmkp2lg297yr0000gr/T/ipykernel_97028/2502697827.py:18: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_test_dummies = pd.get_dummies(df_test, columns=features_dum)
